# 1. Specify all parameters to be varied


In [1]:
from fedt import *

CAD_variables_printer = [{
        'name' : 'side length',
        'data type' : 'ratio',
        'test_values' : [x for x in range(10, 20, 5)],
        'argname' : 'side length'
    }]
CAM_variables_printer = [{
        'name' : 'layer height',
        'data type' : 'ratio',
        'test_values' : [x / 10.0 for x in range(5, 20, 5)], # i hate this so much, but we would need numpy arange to make it better
        'argname' : '--layer-height'
    },
    {
        'name' : 'fill pattern',
        'data type' : 'nominal',
        'test_values' : ['triangles', 'concentric'], # I would like them all to have the same named thing so I can call it
        'argname' : '--fill-pattern'
    }]
CAD_variables_laser = [{
        'name' : 'radius',
        'data type' : 'ratio',
        'test_values' : [x for x in range(10, 20, 5)],
        'argname' : 'radius'
    }]
CAM_variables_laser = [{
        'name' : 'cut power',
        'data type' : 'ratio',
        'test_values' : [x for x in range(5, 100, 10)],
        'argtype' : 'profile'
    },
    {
        'name' : 'cut speed',
        'data type' : 'ratio',
        'test_values' : [x for x in range(5, 100, 10)],
        'argtype' : 'profile'
    }]
fab_repetitions = 2

sample_post_process_variable = {
    'name' : 'paint type',
    'data type' : 'nominal',
    'test_values' : ['lacquer', 'spraypaint'],
    'description' : 'use the {} on the object'
}
post_process_variables = [sample_post_process_variable]
post_process_repetitions = 1

sample_interaction_variable = {
    'name' : 'user push force level',
    'data type' : 'ratio',
    'test_values' : [x for x in range(10)],
    'instruction' : 'push on the button with force {} / 10' 
}
interaction_variables = [sample_interaction_variable]
interaction_repetitions = 1 # TODO: are interaction repetitions ever different from measurement repetitions? I kinda think NO

measurement_variables = [{
        'name' : 'printed x-axis',
        'data type' : 'ratio'
    },
    {
        'name' : 'printed y-axis',
        'data type' : 'ratio'
    },
    {
        'name' : 'printed z-axis',
        'data type' : 'ratio'
    }
]
measurement_repetitions = 5

tea_hypothesis = (['material', 'measured z-axis' ], ['fill_pattern:concentric > triangle'])

###########

experiment_size(CAD_variables_laser, CAM_variables_laser, fab_repetitions,
                post_process_variables, post_process_repetitions,
                interaction_variables,
                measurement_variables, measurement_repetitions)

This experiment will require fabricating 800 unique objects.
Users will perform 40000 interactions.
120000 total measurements will be recorded.


# 2. CAD

In [ ]:
from fedt_print import build_geometry
from fedt_laser import build_geometry

# FreeCAD create cube    
def geometry_function_stl(App, side_length='20 mm'):
    App.ActiveDocument.addObject("Part::Box","Box")
    App.ActiveDocument.ActiveObject.Label = "Cube"
    App.ActiveDocument.recompute()
    FreeCAD.ActiveDocument.getObject('Box').Width = side_length
    FreeCAD.ActiveDocument.getObject('Box').Length = side_length
    FreeCAD.ActiveDocument.getObject('Box').Height = side_length
    App.ActiveDocument.recompute()

def label_function_stl(App, label_string="L0"):
    # https://wiki.freecad.org/Draft_ShapeString_tutorial
    ss=Draft.make_shapestring(String=label_string, FontFile=FONT_LOCATION, Size=5.0, Tracking=0.0)
    plm=FreeCAD.Placement()
    plm.Base=FreeCAD.Vector(4.0, 4.0, side=20)
    plm.Rotation.Q=(0.0, 0.0, 0, 1.0)
    ss.Placement=plm
    ss.Support=None
    Draft.autogroup(ss)
    FreeCAD.ActiveDocument.recompute()
    
    App.ActiveDocument.addObject('Part::Extrusion','Extrude')
    f = App.ActiveDocument.getObject('Extrude')
    f.Base = App.ActiveDocument.getObject('ShapeString')
    f.DirMode = "Normal"
    f.DirLink = None
    f.LengthFwd = 5.000000000000000
    f.LengthRev = 0.000000000000000
    f.Solid = False
    f.Reversed = False
    f.Symmetric = False
    f.TaperAngle = 0.000000000000000
    f.TaperAngleRev = 0.000000000000000
    App.ActiveDocument.recompute()
    App.ActiveDocument.getObject('ShapeString').Visibility = False
    App.ActiveDocument.recompute()
    App.ActiveDocument.getObject('Extrude').Placement = App.Placement(App.Vector(2,2,0),App.Rotation(App.Vector(0,0,1),0))
    App.ActiveDocument.recompute()
    #FreeCAD.ActiveDocument.getObject('Extrude').Shape.exportStl('ext.stl')

# draw a circle in svg
def geometry_function_svg(d, radius=30):
    d.append(draw.Circle(-40, -10, radius,
                fill='none', stroke_width=1, stroke='red'))

def label_function_svg(d, label_string="L0"):
    d.append(draw.Text(x=label_location[0], y=label_location[1], fill='blue', text=label, font_size=10))

# 3. CAM

In [ ]:
from fedt_print import prep_cam, do_cam
from fedt_laser import prep_cam, do_cam

# 4. Fabrication


In [ ]:
from fedt_print import fabricate
from fedt_laser import fabricate

vars_to_labels = {}
i = 0
CAM_paths = []

variables_requiring_labels = CAD_variables_laser + CAM_variables_laser + post_process_variables

for var in variables_requiring_labels:
    # we have to reshape these slightly so that we can build the correct slicing
    

for h in variables['layer height']['values']:
    for p in variables['fill pattern']['categories']:
        for rep in range(fab_repetitions):
            # make a labelled thing
            label = "L" + str(i)
            stl = build_geometry(geometry_function_svg, label_function_svg, label_string=label)
            slicer_settings = {
                '--layer-height': h,
                '--fill-pattern': p
            }
            gcode_path = slice_mesh(stl, slicer_settings)
            i+=1
            CAM_paths.append(gcode_path)
            vars_to_labels[h+"_"+p+"//"+str(rep)] = label

# or send to simplyprint.io for later
for CAM_path in CAM_paths:
    fabricate(CAM_path)

# 5. Post-Processing

In [2]:

# TODO: I was thinking that I need to print out the things to do here, but I actually
# need to attach them somehow to the LABELS in the fabrication step. so that they know what to do to which.
# and then we need to print... something here... so the user knows what to do...

# 6. Create measurement framework

In [4]:
from fedt_measure import create_experiment_csv
experiment_csv = create_experiment_csv(vars_to_labels, measurement_variables, measurement_repetitions)

print("now please fill " + experiment_csv + " with data")

NameError: name 'experiment_csv' is not defined

# 6. User Interaction

In [3]:


# TODO : figure out how to deal with this. is it true that we might need to generate the csv first? I... would think so.

# 8. Tea-based hypothesis testing

In [ ]:
from fedt_tea import do_experiment

tea_results = do_experiment(measurement_variables,
                            tea_hypothesis,
                            experiment_csv,
                            CAD_variables,
                            CAM_variables,
                            post_process_variables,
                            interaction_variables)

# 9. Reporting

In [ ]:
from fedt_report import latex

latex(tea_results,
      tea_hypothesis,
      CAD_variables,
      CAM_variables,
      print_repetitions,
      post_process_variables,
      post_process_repetitions,
      interaction_variables,
      interaction_repetitions,
      measurement_variables,
      measurement_repetitions)